<a href="https://colab.research.google.com/github/Hannah250/PROJECTS/blob/main/Machine_Learning_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.head()

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [5]:
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [6]:
df=df.dropna()
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [7]:
df[ 'QScore' ] = df[ 'QScore' ].replace([ '1A' ], '2A' ) 
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [8]:
df_2A = df[df.QScore== '2A' ] 
df_3A = df[df.QScore== '3A' ].sample( 350 ) 
data_df = df_2A.append(df_3A)

In [9]:
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape
data_df.QScore.value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

In [10]:
data_df.dtypes

country            object
year                int64
country_code        int64
record             object
crop_land         float64
grazing_land      float64
forest_land        object
fishing_ground    float64
built_up_land     float64
carbon            float64
total             float64
QScore             object
dtype: object

In [11]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data_df.record = encoder.fit_transform(data_df.record)

In [12]:
data_df.record

0      7
1      0
2      5
3      6
4      0
      ..
585    0
586    1
587    2
588    0
589    0
Name: record, Length: 590, dtype: int64

In [13]:
data_df = data_df.drop(columns=['country_code', 'country', 'year'])
X = data_df.drop(columns = 'QScore')
y = data_df['QScore']

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
y_train.value_counts()

3A    247
2A    166
Name: QScore, dtype: int64

In [15]:
x_train.record

285    4
113    6
18     7
76     6
206    6
      ..
277    5
9      1
359    1
192    0
559    7
Name: record, Length: 413, dtype: int64

In [16]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_resample(x_train, y_train)
y_train.value_counts()

3A    247
2A    166
Name: QScore, dtype: int64

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record'] = x_train_balanced['record']

In [18]:
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

In [23]:
##Logistic Regression

In [19]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,                   intercept_scaling=1, l1_ratio=None, max_iter=100,                   multi_class='auto', n_jobs=None, penalty='l2',                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,                   warm_start=False)

LogisticRegression()

In [22]:
##Measuring Classification performance

In [20]:
#cross validation score

In [21]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv=5,                     
   scoring='f1_macro')
scores

array([0.49489796, 0.50424119, 0.5533224 , 0.58646617, 0.51428873])

In [24]:
#confusion matrix

In [25]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=['2A', '3A'])
cnf_mat

array([[30, 44],
       [49, 54]])

In [27]:
##Metrics

In [26]:
#Accuracy

In [28]:
accuracy = accuracy_score(y_true = y_test, y_pred = new_predictions)
print('Accuracy: {}'.format(round(accuracy*100, 2)))

Accuracy: 47.46


In [30]:
#Precision

In [29]:
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('Presicion: {}'.format(round(precision*100), 2))

Presicion: 38


In [31]:
#Recall

In [32]:
recall = recall_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('Recall: {}'.format(round(recall*100), 2))

Recall: 41


In [33]:
#F1_score

In [34]:
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('F1: {}'.format(round(f1*100), 2))

F1: 39


In [36]:
#K Fold

In [35]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(normalised_train_df)
f1_scores = []

In [38]:
for train_index, test_index in kf.split(normalised_train_df):  
  x_train, x_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]  
  y_train, y_test = y_balanced[train_index], y_balanced[test_index] 
  model = LogisticRegression().fit(x_train, y_train)   
  f1_scores.append(f1_score(y_true = y_test, y_pred = model.predict(x_test),pos_label = '2A')*100)
f1_scores

[50.90909090909091,
 59.854014598540154,
 51.376146788990816,
 52.23880597014925,
 0.0,
 50.90909090909091,
 59.854014598540154,
 51.376146788990816,
 52.23880597014925,
 0.0]

In [40]:
#Stratified K-Fold

In [39]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
f1_scores = []
for train_index, test_index in skf.split(normalised_train_df, y_balanced):   
  x_train, x_test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]  
  y_train, y_test = y_balanced[train_index], y_balanced[test_index]  
  model = LogisticRegression().fit(x_train, y_train)     
  f1_scores.append(f1_score(y_true = y_test, y_pred = model.predict(x_test),pos_label = '2A')*100)
f1_scores

[64.34782608695652,
 53.46534653465347,
 57.14285714285714,
 48.0,
 68.53146853146853]

In [41]:
#LeaveOneOut

In [42]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo, scoring='f1_macro')
average_score = scores.mean() * 100
average_score

47.97570850202429

In [ ]:
#Tree Based Methods and SVM

In [43]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, y_balanced)

DecisionTreeClassifier()